In [2]:
from floweaver import *
import ipywidgets as widgets
from IPython.display import display, clear_output

size = dict(width=570, height=600)

# Neutral case
dataset_neutral = Dataset.from_csv('neutral-energy-values.csv',
                                   dim_process_filename='new-energy-values-processes.csv')
partition_neutral = dataset_neutral.partition('type')

# Conservative case
dataset_conservative = Dataset.from_csv('conservative-energy-values.csv',
                                       dim_process_filename='new-energy-values-processes.csv')
partition_conservative = dataset_conservative.partition('type')

# Optimistic case
dataset_optimistic = Dataset.from_csv('optimistic-energy-values.csv',
                                      dim_process_filename='new-energy-values-processes.csv')
partition_optimistic = dataset_optimistic.partition('type')

sources = ['Green Hydrogen', 'Blue Hydrogen']
uses = ['Industry', 'Transportation', 'Power', 'Heating']

nodes = {
    'production_sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Production Sources'),
    'Hydrogen Generation': ProcessGroup(['Hydrogen_Generation'], title='Net Hydrogen Generation'),
    'consumption': ProcessGroup('type == "use"', partition=Partition.Simple('process', uses), title='Consumption'),
}

ordering = [
    [['production_sources']],
    [['Hydrogen Generation']],
    [['consumption']],
]

bundles = [
    Bundle('production_sources', 'Hydrogen Generation'),
    Bundle('Hydrogen Generation', 'consumption')
]

palette = {
    'Green Hydrogen': 'green',
    'Blue Hydrogen': 'blue',
}

# Neutral case Sankey
sdd_neutral = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_neutral)
weave(sdd_neutral, dataset_neutral, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

# Conservative case Sankey
sdd_conservative = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_conservative)
weave(sdd_conservative, dataset_conservative, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)

# Optimistic case Sankey
sdd_optimistic = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_optimistic)
weave(sdd_optimistic, dataset_optimistic, palette=palette).to_widget(width=700, height=450, margins=dict(left=100, right=120), debugging=True)
output = widgets.Output()

def on_button_click(button):
    with output:
        clear_output(wait=True)
        if button.description == 'Neutral Case':
            display(weave(sdd_neutral, dataset_neutral, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True))
        elif button.description == 'Conservative Case':
            display(weave(sdd_conservative, dataset_conservative, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True))
        elif button.description == 'Optimistic Case':
            display(weave(sdd_optimistic, dataset_optimistic, palette=palette).to_widget(width=700, height=450, margins=dict(left=150, right=150), debugging=True))

button_neutral = widgets.Button(description="Neutral Case")
button_conservative = widgets.Button(description="Conservative Case")
button_optimistic = widgets.Button(description="Optimistic Case")

button_neutral.on_click(on_button_click)
button_conservative.on_click(on_button_click)
button_optimistic.on_click(on_button_click)

display(widgets.HBox([button_conservative, button_neutral,  button_optimistic]))
display(output)

Output()